###Import library

In [0]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
from pandas.io.json import json_normalize
import folium

###Parameters for API call

In [0]:
Client_ID = 'E52DEM5TQ1OJDFUQP4IG4VZZYIGO1WNN2UX4DCLJTBURTNYG'
Client_secret = 'MEBLWTTDWCJNJJUVF53ICL5XGJ3L3GG445JW4NCT4IGRIFXI'
Version = '20200407'

###API Call

In [0]:
LIMIT = 100
cities = ["Hồ Chí Minh", "Hà Nội"]
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        Client_ID, 
        Client_secret, 
        Version, 
        city,
        LIMIT,
        "4d4b7105d754a06374d81259")
    results[city] = requests.get(url).json()

In [173]:
dfvenues={}
for city in cities:
    venues = json_normalize(results[city]['response']['groups'][0]['items'])
    dfvenues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    dfvenues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


###Visualize on map

In [174]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(dfvenues[city]['Lat'], dfvenues[city]['Lng'], dfvenues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of places to eat in {city} = ", results[city]['response']['totalResults'])

Total number of places to eat in Hồ Chí Minh =  129
Total number of places to eat in Hà Nội =  103


In [175]:
maps[cities[0]]

In [176]:
maps[cities[1]]

### Caculate mean coordinate and mean distance from mean coordinate

In [177]:
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=13)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='red',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.5,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from mean coordinates: ")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Hồ Chí Minh
Mean Distance from mean coordinates: 
0.027077489133903367
Hà Nội
Mean Distance from mean coordinates: 
0.017283118365241025


In [178]:
maps[cities[0]]

In [168]:
maps[cities[1]]